In [85]:
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import load_workbook
import time
import datetime
pd.options.mode.chained_assignment = None  # default='warn'


In [86]:
def create_doc_convert_date(cell):
    """
    Функция для конвертации даты при создании документов
    :param cell:
    :return:
    """
    try:
        string_date = datetime.datetime.strftime(cell, '%d.%m.%Y')
        return string_date
    except ValueError:
        return ''

In [87]:
file_transfer_to_upp = 'data/Перенос из таблицы 3 в реестр УПП/Таблица 3 с дополнительными колонками..xlsx'
file_transfer_reestr = 'data/Перенос из таблицы 3 в реестр УПП/2022-10-27_64_Реестр УПП с дополнительными колонками.xlsx'
path_to_end_folder = 'data/'
header_reestr = 'data/Перенос из таблицы 3 в реестр УПП/Заголовок Реестр УПП от 25.10.xlsx'

In [88]:
# Загружаем датафреймы
df_upp = pd.read_excel(file_transfer_reestr,skiprows=8)
df_table_3 = pd.read_excel(file_transfer_to_upp,skiprows=6,usecols=[4,5,6,7,8,9,10,12,13,14,15,16,17,18,19,21,22,23,24,25,26,27,28,
                                                           29,32])


In [89]:
# Приводим названия колонок к строковому виду, чтобы избежать возможных проблем с названиями колонок
df_upp.columns = list(map(str,list(df_upp.columns)))
df_table_3.columns = list(map(str,list(df_table_3.columns)))

# Очищаем на всякий случай от пробельных символов
df_upp.columns = list(map(lambda x:x.strip(),list(df_upp.columns)))
df_table_3.columns = list(map(lambda x:x.strip(),list(df_table_3.columns)))

In [90]:
# Фильтруем датафрейм отбирая только те записи в которых есть 1 в графе 30
transfer_df=df_table_3[df_table_3['30'] == 1]


# перемещаем площадь выдела
transfer_df.insert(7,'12',transfer_df['33'])

# удаляем лишний столбец с площадью выдела и признаков внесения в реестр
transfer_df.drop(columns=['30','33'],inplace=True)

In [91]:
transfer_df.columns

Index(['5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17',
       '18', '19', '20', '22', '23', '24', '25', '26', '27', '28', '29'],
      dtype='object')

In [92]:
transfer_df['17'] =transfer_df['17'].astype(str) # Приводим колонку к строковому формату

In [93]:
transfer_df['17']

0        134
1        134
2        134
3        134
4        134
        ... 
77603    nan
77610    143
77611    143
77612    143
77613    143
Name: 17, Length: 58813, dtype: object

In [94]:
# Заменяем категории таблицы 3 на категории Реестра УПП
transfer_df['17'] =transfer_df['17'].replace(regex={'120':'2','100':'1',r'130|131|132|133|134|135|136':'3',
                                                    r'141|142|143|144|145|146|147|148|149|150|151|152':'4','nan':'0'})

In [95]:
transfer_df['17']

0        3
1        3
2        3
3        3
4        3
        ..
77603    0
77610    4
77611    4
77612    4
77613    4
Name: 17, Length: 58813, dtype: object

In [96]:
transfer_df.columns

Index(['5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17',
       '18', '19', '20', '22', '23', '24', '25', '26', '27', '28', '29'],
      dtype='object')

In [97]:
transfer_df.columns = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24']

In [98]:
transfer_df.columns

Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24'],
      dtype='object')

In [99]:
# Ищем совпадающие участки
df_upp.head()

,1,2,3,4,5,6,7,8,9,10,...,28,29,30,31,32,33,34,35,36,37
0,Ангоянское,Ангоянское,NaN,32,1,13,1а,0.4,"56,8906071731145","111,604523118265",...,0,0,0.0,0,0.0,NaN,0.4,NaN,NaN,1
1,Ангоянское,Ангоянское,NaN,32,2,7,2а,1.6,"56,8907756254041","111,60655696686",...,0,0,0.0,0,0.0,NaN,1.6,NaN,NaN,1
2,Ангоянское,Ангоянское,NaN,32,3,11,3а,0.1,"56,8853268727696","111,611572021886",...,0,0,0.0,0,0.0,NaN,0.1,NaN,NaN,1
3,Ангоянское,Ангоянское,NaN,32,5,10,5а,7.5,"56,8885191270788","111,601674714733",...,0,0,0.0,0,0.0,NaN,7.5,NaN,NaN,1
4,Ангоянское,Ангоянское,NaN,32,7,78,7а,20.5,"56,8783556985033","111,572764198335",...,0,0,0.0,0,0.0,NaN,20.5,NaN,NaN,1


In [100]:
transfer_df.head()

,1,2,3,4,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
0,Ангоянское,Ангоянское,NaN,32,1,13.0,NaN,0.0,NaN,NaN,...,0,21,2022-02-04 00:00:00,0,0,0,0,0,0,0
1,Ангоянское,Ангоянское,NaN,32,1,300.0,NaN,0.0,NaN,NaN,...,0,21,2022-02-04 00:00:00,0,0,0,0,0,0,0
2,Ангоянское,Ангоянское,NaN,32,1,500.0,NaN,0.0,NaN,NaN,...,0,21,2022-02-04 00:00:00,0,0,0,0,0,0,0
3,Ангоянское,Ангоянское,NaN,32,1,400.0,NaN,2.3,NaN,NaN,...,0,21,2022-02-04 00:00:00,1,СО,более 40%,более 40%,2022-04-05 00:00:00,1,0
4,Улан-Удэнское,Березовское,NaN,39,8,2.0,NaN,0.8,NaN,NaN,...,0,21,2022-02-04 00:00:00,1,СО,более 40%,более 40%,2022-04-05 00:00:00,1,0


In [119]:
# Создаем датафрейм для проверки наличия записи в реестре
checked_df = df_upp[['1','2','3','4','5']]
checked_df = checked_df.astype(str) # делаем данные строковыми
# Создаем в каждом датафрейме колонку с айди путем склеивания всех нужных колонок в одну строку
checked_df['ID'] = checked_df.loc[:,['1','2','3','4','5'] ].sum(axis=1)
# Удаляем пробелы
checked_df['ID'] = checked_df['ID'].apply(lambda x:x.replace(' ',''))


In [120]:
# делаем строковыми первые 5 колонок
transfer_df[['1','2','3','4','5',]] = transfer_df[['1','2','3','4','5',]].astype(str)
transfer_df['ID'] = transfer_df.loc[:,['1','2','3','4','5'] ].sum(axis=1)
transfer_df['ID'] = transfer_df['ID'].apply(lambda x:x.replace(' ',''))

In [121]:
# Мержим по полю айди
merged_df = pd.merge(checked_df,transfer_df,how='outer',left_on='ID',right_on='ID',indicator=True)

In [124]:
# Отбираем только те значения которые есть в правом датафрейме
added_df = merged_df[merged_df['_merge']=='right_only']

In [126]:
added_df.head()

,1_x,2_x,3_x,4_x,5_x,ID,1_y,2_y,3_y,4_y,...,16,17,18,19,20,21,22,23,24,_merge
405310,NaN,NaN,NaN,NaN,NaN,ЕравнинскоеГундинское2311,Еравнинское,Гундинское,,23,...,21.0,2022-03-09 00:00:00,1.0,БКС,более 40%,более 40%,2022-07-19 00:00:00,1.0,0.0,right_only
405311,NaN,NaN,NaN,NaN,NaN,ЕравнинскоеГундинское2311,Еравнинское,Гундинское,,23,...,21.0,2022-03-09 00:00:00,1.0,БКС,более 40%,более 40%,2022-07-19 00:00:00,1.0,0.0,right_only
405312,NaN,NaN,NaN,NaN,NaN,ЕравнинскоеГундинское316,Еравнинское,Гундинское,,31,...,21.0,2022-03-09 00:00:00,1.0,БКС,более 40%,более 40%,2022-07-19 00:00:00,1.0,0.0,right_only
405313,NaN,NaN,NaN,NaN,NaN,ЕравнинскоеГундинское316,Еравнинское,Гундинское,,31,...,21.0,2022-03-09 00:00:00,1.0,БКС,более 40%,более 40%,2022-07-19 00:00:00,1.0,0.0,right_only
405314,NaN,NaN,NaN,NaN,NaN,ЕравнинскоеГундинское3115,Еравнинское,Гундинское,,31,...,21.0,2022-03-09 00:00:00,1.0,БКС,более 40%,более 40%,2022-07-19 00:00:00,1.0,0.0,right_only


In [128]:
added_df.drop(columns=['1_x','2_x','3_x','4_x','5_x','ID','_merge'],inplace=True) # удаляем лишние колонки

In [130]:
added_df.rename(columns={'1_y':'1','2_y':'2','3_y':'3','4_y':'4','5_y':'5'},inplace=True) # переименовываем колонки для корректного добавления

In [131]:
itog_df = pd.concat([df_upp,added_df],ignore_index=True)

In [132]:
itog_df.sort_values(['1','2','3','4','5',
                     ],inplace=True)

In [133]:
itog_df.head()

,1,2,3,4,5,6,7,8,9,10,...,28,29,30,31,32,33,34,35,36,37
0,Ангоянское,Ангоянское,NaN,32,1,13,1а,0.4,"56,8906071731145","111,604523118265",...,0,0,0.0,0,0.0,NaN,0.4,NaN,NaN,1.0
1,Ангоянское,Ангоянское,NaN,32,2,7,2а,1.6,"56,8907756254041","111,60655696686",...,0,0,0.0,0,0.0,NaN,1.6,NaN,NaN,1.0
2,Ангоянское,Ангоянское,NaN,32,3,11,3а,0.1,"56,8853268727696","111,611572021886",...,0,0,0.0,0,0.0,NaN,0.1,NaN,NaN,1.0
3,Ангоянское,Ангоянское,NaN,32,5,10,5а,7.5,"56,8885191270788","111,601674714733",...,0,0,0.0,0,0.0,NaN,7.5,NaN,NaN,1.0
4,Ангоянское,Ангоянское,NaN,32,7,78,7а,20.5,"56,8783556985033","111,572764198335",...,0,0,0.0,0,0.0,NaN,20.5,NaN,NaN,1.0


In [103]:
# Приводим даты к нормальному виду ДД.ММ.ГГГГ
itog_df['17'] = pd.to_datetime(itog_df['17'],errors='coerce',dayfirst=True)
itog_df['22'] = pd.to_datetime(itog_df['22'],errors='coerce',dayfirst=True)
itog_df['17'] = itog_df['17'].apply(create_doc_convert_date)
itog_df['22'] = itog_df['22'].apply(create_doc_convert_date)


In [134]:
# Получаем текущую дату
current_time = time.strftime('%H_%M_%S %d.%m.%Y')
# Сохраняем отчет
# Для того чтобы увеличить ширину колонок для удобства чтения используем openpyxl
wb = load_workbook(header_reestr) # Создаем объект
# Записываем результаты
for row in dataframe_to_rows(itog_df,index=False,header=False):
    wb['Реестр УПП'].append(row)

In [ ]:
wb.save(f'{path_to_end_folder}/Реестр УПП с добавлением данных из таблицы 3 {current_time}.xlsx')